Read two saved template Models 
Interpolate Ge and Ph data to create input and output targets for training.  

1. Load existing template NN models
2. Create parameters for generating training data from source template models
3. Initialize template interpolation (ti) model
4. Train ti model
5. Generate test data for graphing
6. Save ti model to file
7. Generate blender data from ti model output

In [2]:
import torch as torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import os
import numpy as np
import json 
import math as m
import random
import time
import NNArt as gy
import NNArt1 as gy1
from sympy.utilities.iterables import multiset_permutations

import plotly as py
import plotly.offline as offline
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.io as pio

CURRENT_MODEL_SAVE_DIR = './save/'

offline.init_notebook_mode(connected=True) 
np.set_printoptions(precision=3, suppress=True)

In [3]:
class Net(nn.Module):
    def __init__(self, layerSizes, name):
        super(Net, self).__init__()
        self.shape = layerSizes
        self.name = name
        self.seq = nn.Sequential(
            nn.Linear(layerSizes[0],layerSizes[1]),
            nn.ELU(),          
            nn.Linear(layerSizes[1],layerSizes[2]),
            nn.ELU(),            
            nn.Linear(layerSizes[2],layerSizes[3]),
            nn.ELU()
        )          
    def forward(self, x):
        return self.seq(x) 

class NetEEL(nn.Module):
    def __init__(self, layerSizes, name):
        super(NetEEL, self).__init__()
        self.shape = layerSizes
        self.name = name
        self.seq = nn.Sequential(
            nn.Linear(layerSizes[0],layerSizes[1]),
            nn.ELU(),          
            nn.Linear(layerSizes[1],layerSizes[2]),
            nn.ELU(),            
            nn.Linear(layerSizes[2],layerSizes[3]),
            nn.ELU(),            

        )          
    def forward(self, x):
        return self.seq(x) 

In [7]:
'''
1. Load existing template NN models
templateModel2

'''

tmModel0 = Net([2,10,15,21],'tmModel0')
tmModel1 = Net([2,10,15,21],'tmModel1')
relModel = Net([21,15,10,3],'relModel')
genModel = Net([3,10,15,21],'genModel')
cohModel = Net([3,5,3,2],'cohModel')

PATH=CURRENT_MODEL_SAVE_DIR+'templateModel0.pth'
tmModel0.load_state_dict(torch.load(PATH))
PATH=CURRENT_MODEL_SAVE_DIR+'templateModel1.pth'
tmModel1.load_state_dict(torch.load(PATH))
PATH=CURRENT_MODEL_SAVE_DIR+'REL PhRc-T0T1-mask 11p05.pth'
relModel.load_state_dict(torch.load(PATH))
PATH=CURRENT_MODEL_SAVE_DIR+'GEN RcGen-T0T1-mask 11p05.pth'
genModel.load_state_dict(torch.load(PATH))
PATH=CURRENT_MODEL_SAVE_DIR+'cohModel.pth'
cohModel.load_state_dict(torch.load(PATH))
tmModel0.eval()
tmModel1.eval()
relModel.eval()
genModel.eval()
cohModel.eval()

tmModel0.shape, tmModel1.shape,genModel.shape

([2, 10, 15, 21], [2, 10, 15, 21], [3, 10, 15, 21])

In [8]:
'''
4. Review Emergent Context 


'''
morphIndices=[0.,0.1,.2,.3,.5,.7,.9,1.0, 1.1]
timeIndices=[0.,0.1,.2,.3,.5,.7,.9,1.0, 1.1]

#sample source templates to get ph samples
relIn = gy.createRmInput(morphIndices, timeIndices, sPermutation=[])
tmOut0 = tmModel0(torch.Tensor(relIn))
tmOut1 = tmModel1(torch.Tensor(relIn))

#use template generated ph samples to feed through contex models
relCout0 = relModel(tmOut0)
relCout1 = relModel(tmOut1)

genOut0 = genModel(relCout0)
genOut1 = genModel(relCout1)

cohOut0 = cohModel(relCout0)
cohOut1 = cohModel(relCout1)

relIn.shape, tmOut0.shape, relCout0.shape, genOut0.shape, cohOut0.shape

((9, 9, 2),
 torch.Size([9, 9, 21]),
 torch.Size([9, 9, 3]),
 torch.Size([9, 9, 21]),
 torch.Size([9, 9, 2]))

In [9]:
'''
Build xy values for 
Context values over morphology for different time values

'''

#relCout0 for 0-8 time intervals
gTp0T0 = np.array([relIn[0,:,1],relCout0[0,:,2].detach().numpy()])
gTp0Tp2 = np.array([relIn[2,:,1],relCout0[2,:,2].detach().numpy()])
gTp0Tp5 = np.array([relIn[4,:,1],relCout0[4,:,2].detach().numpy()])
gTp0Tp9 = np.array([relIn[6,:,1],relCout0[6,:,2].detach().numpy()])
gTp0T1 = np.array([relIn[7,:,1],relCout0[7,:,2].detach().numpy()])
gTp0T1p1 = np.array([relIn[8,:,1],relCout0[8,:,2].detach().numpy()])

#relCout1 for 0-8 time intervals
gTp1T0 = np.array([relIn[0,:,1],relCout1[0,:,2].detach().numpy()])
gTp1Tp2 = np.array([relIn[2,:,1],relCout1[2,:,2].detach().numpy()])
gTp1Tp5 = np.array([relIn[4,:,1],relCout1[4,:,2].detach().numpy()])
gTp1Tp9 = np.array([relIn[6,:,1],relCout1[6,:,2].detach().numpy()])
gTp1T1 = np.array([relIn[7,:,1],relCout1[7,:,2].detach().numpy()])
gTp1T1p1 = np.array([relIn[8,:,1],relCout1[8,:,2].detach().numpy()])

gx = [gTp0T0, gTp1T0, gTp0Tp2, gTp1Tp2, gTp0Tp5, gTp1Tp5,
    gTp0Tp9, gTp1Tp9,gTp0T1p1,gTp1T1p1]


In [10]:
layout = go.Layout(
    title = 'Initial Curves',
    height = 700,
    width = 800,
    hovermode= 'closest',
    xaxis= dict(
        title= 'Morphology',
        range = [-.1,1.5],
    ),
    yaxis=dict(
        title= 'Context',
        range = [-.6,1.],
    )
)
layout.title = 'Context values over morphology for different time values' 
f = gy.graphCurves(gx, layout, labels=['gTp0T0','gTp1T0', 'gTp0Tp2', 'gTp1Tp2', 'gTp0T0p5', 'gTp1T0p5', 
                                       'gTp0T0p9','gTp1T0p9',  'gTp0T0_1p1','gTp1T0_1p1'], 
                   dashRange=gy.rangeList([range(2,4),range(6,8)]))

offline.iplot(f)


In [13]:
'''
Format for graphing at a time point

relIn[t,:,1]    is X axis m values, 
relCout0[t,:,2] is y axis is context values for 0 template


'''

t=8
#structure context values for graphing
con0 = np.array([relIn[t,:,1],relCout0[t,:,2].data.numpy()]) 
con1 = np.array([relIn[t,:,1],relCout1[t,:,2].data.numpy()])

#coherence values
#reshape into a 1,1,9 list of 9 coh values
# ch0 = np.reshape(cohOut0[t].data.numpy(), [1,2,-1])
# ch1 = np.reshape(cohOut1[t].data.numpy(), [1,2,-1])
ch0 = np.reshape(cohOut1[t].data.numpy()[:,0],[1,1,-1])
coh0 = np.array([relIn[t,:,1],ch0[0,0]])
cohD0 =  np.array(con0)
cohD0[1,:] = coh0[1,:]+con0[1,:]

In [12]:
# '''
# Format for graphing at a time point

# relIn[t,:,1]    is X axis m values, 
# relCout0[t,:,2] is y axis is context values for 0 template


# '''

# t=8
# #structure context values for graphing
# con0 = np.array([relIn[t,:,1],relCout0[t,:,2].data.numpy()]) 
# con1 = np.array([relIn[t,:,1],relCout1[t,:,2].data.numpy()])

# #coherence values
# #reshape into a 1,1,9 list of 9 coh values
# ch0 = np.reshape(cohOut0[t].data.numpy(), [1,1,-1])
# ch1 = np.reshape(cohOut1[t].data.numpy(), [1,1,-1])
# #add m values for x axis
# coh0 = np.array([relIn[t,:,1],ch0[0,0]])
# coh1 = np.array([relIn[t,:,1],ch1[0,0]])

# #cohD adds coh t0 context
# cohD0 =  np.array(con0)
# cohD1 =  np.array(con1)
# cohD0[1,:] = coh0[1,:]+con0[1,:]
# cohD1[1,:] = coh1[1,:]+con1[1,:]


In [14]:
gxx = [con0,con1,coh0,cohD0]
layout = go.Layout(
    title = 'Initial Curves',
    height = 600,
    width = 600,
    hovermode= 'closest',
    xaxis= dict(
        title= 'X',
        range = [-.4,1.2],
    ),
    yaxis=dict(
        title= 'Y',
        range = [-.4,.5],
    )
)
fxx = gy.graphCurves(gxx, layout, labels=['context0','context1','coherence0','coherence1', 'con0+coh0','con1+coh1'], dashRange=range(0,2))
offline.iplot(fxx)

In [15]:
#create a matrix of mesh values
mm,cc = np.meshgrid(np.arange(-0., 1.1, .1), np.arange(-.5, .9, .1))

t = .9
ma = np.array(mm)
ca = np.array(cc)
xList = []
yList = []
uList = []
vList = []
for mi in range(ma.shape[0]):
    for ci in range(ca.shape[1]):
        cIn = torch.Tensor([[t,ma[mi,ci], ca[mi,ci]]])
        cohOut = cohModel(cIn)
        x = ma[mi,ci]
        y = ca[mi,ci]
        u = 0
        v = cohOut.data.numpy()[0,0] #extract value 0.0716
        xList.append(x)
        yList.append(y)
        uList.append(u)
        vList.append(v)

xx = np.array(xList)
yy = np.array(yList)
uu = np.array(uList)
vv = np.array(vList)

fig0 = ff.create_quiver(xx, yy, uu, vv,
       scale=.4,
       arrow_scale=.4,
       name='quiver'
       #,line=dict(width=2)
      )

fig0.layout.title = 'Coherence field for time ' + str(t)
fig0.add_trace(f.data[6]) 
fig0.add_trace(f.data[7]) 
offline.iplot(fig0)
layout.title

layout.Title({
    'text': 'Initial Curves'
})

In [16]:
#create a matrix of mesh values
mm,cc = np.meshgrid(np.arange(-0., 1.1, .1), np.arange(-.5, .9, .1))

t = .5
ma = np.array(mm)
ca = np.array(cc)
xList = []
yList = []
uList = []
vList = []
for mi in range(ma.shape[0]):
    for ci in range(ca.shape[1]):
        cIn = torch.Tensor([[t,ma[mi,ci], ca[mi,ci]]])
        cohOut = cohModel(cIn)
        x = ma[mi,ci]
        y = ca[mi,ci]
        u = 0
        v = cohOut.data.numpy()[0,0] #extract value 0.0716
        xList.append(x)
        yList.append(y)
        uList.append(u)
        vList.append(v)

xx = np.array(xList)
yy = np.array(yList)
uu = np.array(uList)
vv = np.array(vList)

fig0 = ff.create_quiver(xx, yy, uu, vv,
       scale=.4,
       arrow_scale=.4,
       name='quiver'
       #,line=dict(width=2)
      )

fig0.layout.title = 'Coherence field for time ' + str(t)
fig0.add_trace(f.data[4]) 
fig0.add_trace(f.data[5]) 
offline.iplot(fig0)
layout.title

layout.Title({
    'text': 'Initial Curves'
})

In [17]:
#create a matrix of mesh values
mm,cc = np.meshgrid(np.arange(-0., 1.2, .1), np.arange(-.5, 1, .1))

t = .2
ma = np.array(mm)
ca = np.array(cc)
xList = []
yList = []
uList = []
vList = []
for mi in range(ma.shape[0]):
    for ci in range(ca.shape[1]):
        cIn = torch.Tensor([[t,ma[mi,ci], ca[mi,ci]]])
        cohOut = cohModel(cIn)
        x = ma[mi,ci]
        y = ca[mi,ci]
        u = 0
        v = cohOut.data.numpy()[0,0] #extract value 0.0716
        xList.append(x)
        yList.append(y)
        uList.append(u)
        vList.append(v)


xx = np.array(xList)
yy = np.array(yList)
uu = np.array(uList)
vv = np.array(vList)

fig0 = ff.create_quiver(xx, yy, uu, vv,
       scale=.4,
       arrow_scale=.4,
       name='quiver'
       #,line=dict(width=2)
      )

fig0.layout.title = 'Coherence field for time ' + str(t)
fig0.add_trace(f.data[2]) 
fig0.add_trace(f.data[3]) 
offline.iplot(fig0)
layout.title

layout.Title({
    'text': 'Initial Curves'
})

In [63]:
#create a matrix of mesh values
mm,cc = np.meshgrid(np.arange(-0., 1.1, .1), np.arange(-.8, 1, .1))

t = .0

ma = np.array(mm)
ca = np.array(cc)
xList = []
yList = []
uList = []
vList = []
for mi in range(ma.shape[0]):
    for ci in range(ca.shape[1]):
        cIn = torch.Tensor([[t,ma[mi,ci], ca[mi,ci]]])
        cohOut = cohModel(cIn)
        x = ma[mi,ci]
        y = ca[mi,ci]
        u = 0
        v = cohOut.data.numpy()[0,0] #extract value 0.0716
        xList.append(x)
        yList.append(y)
        uList.append(u)
        vList.append(v)


xx = np.array(xList)
yy = np.array(yList)
uu = np.array(uList)
vv = np.array(vList)

fig0 = ff.create_quiver(xx, yy, uu, vv,
       scale=.3,
       arrow_scale=.2,
       name='quiver'
       #,line=dict(width=2)
      )

fig0.layout.title = 'Coherence field for time ' + str(t)
fig0.add_trace(f.data[0]) 
fig0.add_trace(f.data[1]) 
offline.iplot(fig0)
layout.title

layout.Title({
    'text': 'Initial Curves'
})

In [170]:
##################
tindex=6 #.9
cohMult = [.1,.08, .06,.05,.05,.02,.01] #9.2
cohMult = [.1,.1, .08,.05,.05,.02,.01] #9
cohMult = [.06,.06,.06,.05,.05,.05,.05] #8.5
cohMult = [.09,.08, .06,.05,.05,.02,.02] #8
cohMult = [.15,.1, .08,.06,.05,.04,.01] #8
cohMult = [.1,.1,.08,.06,.05,.05,.04] #7

cohMult = [.1,.07, .06,.05,.04,.02,.01, .01] #7
cohMult = [.09,.07, .06,.05,.04,.02,.01, .01] #9
cohMult = [.1,.08, .06,.05,.05,.02,.01, .01] #9.3

cohMult = [.1,.1, .1,.1,.02,.02,.01, .01] #9.3

#create a random input
relIn = torch.Tensor(gy.createRmInput(morphIndices, timeIndices, sPermutation=[]))
genIn = torch.Tensor(np.random.random([1,9,3]))

relIn[6].shape

torch.Size([9, 2])

In [182]:
#target 
morphIndices=[0.,0.1,.2,.3,.5,.7,.9,1.0, 1.1]
timeIndices=[0.,0.1,.2,.3,.5,.7,.9,1.0, 1.1]

#sample source templates to get ph samples
relIn1 = gy.createRmInput(morphIndices, [.9], sPermutation=[])
tmOut0 = tmModel0(torch.Tensor(relIn1))
relOut0 = relModel(tmOut0)

##target C values
relOut0

tensor([[[ 0.9553, -0.0019,  0.2053],
         [ 0.9269,  0.1252,  0.1669],
         [ 0.9303,  0.2330,  0.1373],
         [ 0.9292,  0.3261,  0.1097],
         [ 0.9239,  0.5061,  0.0555],
         [ 0.9058,  0.6967, -0.0018],
         [ 0.8912,  0.9142, -0.0719],
         [ 0.8851,  1.0384, -0.1176],
         [ 0.8720,  1.1683, -0.1716]]], grad_fn=<EluBackward>)

In [183]:
relCout2

tensor([[[0.9000, 0.0000, 0.3252],
         [0.9000, 0.1000, 0.1192],
         [0.9000, 0.2000, 0.3855],
         [0.9000, 0.3000, 0.3413],
         [0.9000, 0.5000, 0.4595],
         [0.9000, 0.7000, 0.2480],
         [0.9000, 0.9000, 0.4136],
         [0.9000, 1.0000, 0.3201],
         [0.9000, 1.1000, 0.2919]]], grad_fn=<CopySlices>)

In [184]:
cohMult[0], cohOut2[:,:,0],relCout2a

(0.1,
 tensor([[-0.0260,  0.0343, -0.0675, -0.0603, -0.1625, -0.0780, -0.2444, -0.1904,
          -0.1951]], grad_fn=<SelectBackward>),
 tensor([[[0.9000, 0.0000, 0.3226],
          [0.9000, 0.1000, 0.1227],
          [0.9000, 0.2000, 0.3788],
          [0.9000, 0.3000, 0.3353],
          [0.9000, 0.5000, 0.4433],
          [0.9000, 0.7000, 0.2402],
          [0.9000, 0.9000, 0.3892],
          [0.9000, 1.0000, 0.3011],
          [0.9000, 1.1000, 0.2724]]], grad_fn=<CopySlices>))

In [180]:
relCout2-relCout2a

tensor([[[ 0.0000,  0.0000,  0.0026],
         [ 0.0000,  0.0000, -0.0034],
         [ 0.0000,  0.0000,  0.0067],
         [ 0.0000,  0.0000,  0.0060],
         [ 0.0000,  0.0000,  0.0163],
         [ 0.0000,  0.0000,  0.0078],
         [ 0.0000,  0.0000,  0.0244],
         [ 0.0000,  0.0000,  0.0190],
         [ 0.0000,  0.0000,  0.0195]]], grad_fn=<SubBackward0>)

In [167]:
cohMult[0],relCout2a

(0.1, tensor([[[0.9000, 0.0000, 0.4954],
          [0.9000, 0.1000, 0.1019],
          [0.9000, 0.2000, 0.4173],
          [0.9000, 0.3000, 0.4035],
          [0.9000, 0.5000, 0.4899],
          [0.9000, 0.7000, 0.4686],
          [0.9000, 0.9000, 0.0827],
          [0.9000, 1.0000, 0.4524],
          [0.9000, 1.1000, 0.2131]]], grad_fn=<CopySlices>))

In [177]:
genOut1 = genModel(genIn) #random ph1

relCout1 = relModel(genOut1) #rel
relCout1[0,:,0:2] = relIn[tindex] #adjust t,m
genOut2 = genModel(relCout1) #ph2

cohOut1 = cohModel(relCout1)
relCout1a = relCout1.clone().detach() #clone
relCout1a[0:1,:,2] += cohOut1[:,:,0]*cohMult[0] #coherence adjusted context
genOut2a = genModel(relCout1a) #ph2a

relCout2 = relModel(genOut2a) #rel
relCout2[0,:,0:2] = relIn[tindex] #adjust t,m
genOut3 = genModel(relCout2) #ph3

cohOut2 = cohModel(relCout2)
relCout2a = relCout2.clone().detach() #clone
relCout2a[0:1,:,2] += cohOut2[:,:,0]*cohMult[1] #coherence adjusted context
genOut3a = genModel(relCout2a) #ph3a


In [138]:
ph0 = genOut1.data.numpy()[0]
ph1 = genOut2.data.numpy()[0]
ph2 = genOut2a.data.numpy()[0]
ph3 = genOut3.data.numpy()[0]
ph4 = genOut3a.data.numpy()[0]
# ph5 = genOut6.data.numpy()[0]
# ph6 = genOut7.data.numpy()[0]
# ph7 = genOut8.data.numpy()[0]
# ph8 = genOut9.data.numpy()[0]
# ph9 = genOut10.data.numpy()[0]
# ph10 = genOut11.data.numpy()[0]
# ph11 = genOut12.data.numpy()[0]
# ph12 = genOut13.data.numpy()[0]


# nx = np.concatenate((ph1,ph1))
# nx = np.concatenate((ph0,ph12))
nx = np.concatenate((ph0,ph4))

gx = gy.FormatNetworkToGraphColumns(nx, xOffset=0, yOffset=14, colPoints=7)

layout = go.Layout(
    title = 'Initial Curves',
    height = 600,
    width = 600,
    hovermode= 'closest',
    xaxis= dict(
        title= 'X',
        range = [-.8,1.2],
    ),
    yaxis=dict(
        title= 'Z',
        range = [-.2,.8],
    )
)
f = gy.graphCurves(gx, layout, dashRange=range(0,9))
offline.iplot(f)


In [139]:
gx = gy.FormatNetworkToGraphColumns(nx, xOffset=7, yOffset=14, colPoints=7)

layout = go.Layout(
    title = 'Initial Curves',
    height = 600,
    width = 600,
    hovermode= 'closest',
    xaxis= dict(
        title= 'Y',
        range = [-1.,1.2],
    ),
    yaxis=dict(
        title= 'Z',
        range = [-.2,.8],
    )
)
f = gy.graphCurves(gx, layout, dashRange=range(0,9))
offline.iplot(f)

In [140]:
cohMult = [.06,.06,.06,.05,.05,.05,.05] #8.5

gx = gy.FormatNetworkToGraphColumns(nx, xOffset=0, yOffset=7, colPoints=7)

layout = go.Layout(
    title = 'Initial Curves',
    height = 600,
    width = 600,
    hovermode= 'closest',
    xaxis= dict(
        title= 'X',
        range = [-.6,1],
    ),
    yaxis=dict(
        title= 'Y',
        range = [-1.3,1.3],
    )
)
f = gy.graphCurves(gx, layout, dashRange=range(0,9))
offline.iplot(f)

In [126]:
cohMult = [.15,.1, .08,.06,.05,.04,.01] #8

gx = gy.FormatNetworkToGraphColumns(nx, xOffset=0, yOffset=7, colPoints=7)

layout = go.Layout(
    title = 'Initial Curves',
    height = 600,
    width = 600,
    hovermode= 'closest',
    xaxis= dict(
        title= 'X',
        range = [-.6,1],
    ),
    yaxis=dict(
        title= 'Y',
        range = [-1.3,1.3],
    )
)
f = gy.graphCurves(gx, layout, dashRange=range(0,9))
offline.iplot(f)

In [ ]:
'''
9. Generate blender data from template interpolation model output

#phOut.shape torch.Size([10, 7, 23])

'''

# phOut = phModel(torch.Tensor(torch.Tensor(inpt0)))

# x = phOut.detach().numpy()
# fb = gy.FormatNetworkToBlender(x, xOffset=0, yOffset=7, zOffset=14, colPoints=7)

# with open(CURRENT_MODEL_SAVE_DIR +'btest.json', 'w') as fp:
#     json.dump(fb.tolist(),fp)
    
# x.shape, fb.shape